In [1]:
from datasets import load_dataset

reference = load_dataset("roneneldan/TinyStories")

/Users/maxmynter/Library/Caches/pypoetry/virtualenvs/wholesomegreentextllm-RqUaalAq-py3.10/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [2]:
import pandas as pd

In [3]:
#gtx = pd.read_csv("../../data/open_ai_110k.csv")
#gtx = pd.read_csv("../simple_generation/generated_simple_greentexts.csv")
gtx = pd.read_csv("../../data/openai_thanks_59k.csv")

ref = pd.DataFrame(reference['train'])

print(gtx.head())
print(ref.head())

   Unnamed: 0                                         greentexts
0           0  > Meet new friend at playground\n> Share toys ...
1           1  > Have playdate at Grandma's house\n> Grandma ...
2           2  > Go to birthday party\n> Get a present from a...
3           3  > Help Mom make dinner\n> Stir the sauce and a...
4           4  > Go to school\n> Teacher reads a fun story\n>...
                                                text
0  One day, a little girl named Lily found a need...
1  Once upon a time, there was a little car named...
2  One day, a little fish named Fin was swimming ...
3  Once upon a time, in a land full of trees, the...
4  Once upon a time, there was a little girl name...


In [4]:
def word_process(word):
    word = word.replace("'","") # Remove double string quotation marks
    punctuation = ",.!?;:'%"
    for p in punctuation:
        word = word.replace(p,"")
    return word.lower()

In [5]:
ref_set = set()

for index, row in ref.iterrows():
    [ref_set.add(word_process(word)) for word in row["text"].split()]
    

In [6]:
add_words = [
    "exe", "jpg", "pepe", "mfw", "meme", "4chan", "twitter", "instagram", "facebook", 'youtube'
    "tiktok", "feelsbadman", "discord", "anon", "google", "netflix", "iphone", "", "...",
    "caesar", "computer", "pc", "nietzsche", "cleopatra", "mapoleon", "bonaparte", "bitcoin",
    "cryptocurrency", "crypto", "buffering", "spam", "download", "keyboard", "lol"
]

for w in add_words:
    ref_set.add(w)
print(len(ref_set))

100300


In [7]:
# Idea. Maybe want to still allow certain selected concepts to still exist even though they are not simple words. 
# Like .exe, .jpg, pepe, mfw, meme, 4chan, twitter, instagram, facebook, tiktok, Feelsbadman, discord, anon, google, netflix 
# Should include '...', 
# (Idea) Can also, for simplification, make everything to lowercase. 
# In any case, make the set lower such that capitalization does not matter for word ID

In [8]:
gtx["is_simple"] = 0

for index, row in gtx.iterrows():
    greentext_words = row["greentexts"].replace("\n", "" ).replace(">", " ").split()
    greentext_words = [word_process(word) for word in greentext_words]
    if set(greentext_words).issubset(ref_set):
        gtx.at[index, "is_simple"] = 1



In [9]:
simple_gtx = gtx[gtx["is_simple"] == 1]
hard_gtx = gtx[gtx["is_simple"] == 0]

In [10]:
simple_gtx

,Unnamed: 0,greentexts,is_simple
0,0,> Meet new friend at playground\n> Share toys ...,1
1,1,> Have playdate at Grandma's house\n> Grandma ...,1
2,2,> Go to birthday party\n> Get a present from a...,1
3,3,> Help Mom make dinner\n> Stir the sauce and a...,1
4,4,> Go to school\n> Teacher reads a fun story\n>...,1
...,...,...,...
58748,59455,> Question the concept of time\n> Wonder if it...,1
58749,59456,> Ponder about the meaning of love\n> Feel war...,1
58751,59458,> Reflect on the purpose of laughter\n> Releas...,1
58752,59459,> Wonder about the power of words\n> Realize t...,1


In [11]:
for idx, row in gtx.iterrows():
    if not ">" in row["greentexts"]:
        print(row["greentexts"])

In [12]:
# Only uncomment and execute if you really want to save something
simple_gtx['greentexts'].to_csv("openai_simple_thanks.csv")

In [13]:
hard_gtx

,Unnamed: 0,greentexts,is_simple
30,30,> Grandma gives me a new smartwatch\n> Take a ...,0
58,58,> Uncle downloads a fun game on the tablet\n> ...,0
66,66,> Find Mom's old iPod in the drawer\n> Listen ...,0
95,95,> Going on a train ride\n> Spot a steam engine...,0
105,105,> Uncle lets me play with his VR headset\n> Ex...,0
...,...,...,...
58676,59356,> Build a tall tower with blocks\n> Step back ...,0
58720,59427,> Visited a faraway planet\n> Met friendly ali...,0
58726,59433,> Built a time-traveling machine\n> Visited th...,0
58727,59434,> Went on an adventure to the moon\n> Explored...,0
